## Libraries

In [1]:
#  Install required packages
!pip install -U -q transformers accelerate peft bitsandbytes
!pip install -U -q git+https://github.com/huggingface/trl.git
!pip install -q evaluate
!pip install -q sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [13]:
#  Imports
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
from trl import SFTTrainer
import torch
import math
from tqdm import tqdm

##Dataset

In [4]:
# ✅ Load QnA dataset from plain .txt files
def load_qna(path):
    with open(path, "r", encoding="utf-8") as f:
        blocks = f.read().strip().split("\n\n")  # Two newlines = one QnA pair
    return [{"text": block.strip()} for block in blocks]

train_dataset = Dataset.from_list(load_qna("Train.txt"))
val_dataset   = Dataset.from_list(load_qna("Validation.txt"))

print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))
print("Example:", train_dataset[0]["text"])

Train samples: 800
Val samples: 100
Example: Q: How do I get in touch with the company?
A: You can email us directly at unthealthandfood@gmail.com for any help or questions.


In [5]:
# ✅ Load GPT-2-Large with 4-bit quantization
model_name = "gpt2-large"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

## Model building

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model = prepare_model_for_kbit_training(model)

# avoiding padding issues
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
#  Apply LoRA for QLoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [8]:
#  Training setup
training_args = TrainingArguments(
    output_dir="./gpt2-qlora-output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_total_limit=2,
    save_steps=100,
    report_to="none"
)

In [9]:
#  Fine-tune using SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=training_args,
    formatting_func=lambda x: x["text"],
    peft_config=lora_config
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'gpt2-large' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [10]:
#Train model
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.526600
20,2.825900
30,2.382900
40,1.864800
50,1.532000
60,1.196300
70,0.986300
80,0.828500
90,0.726600
100,0.606500


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=300, training_loss=0.7549880544344584, metrics={'train_runtime': 484.977, 'train_samples_per_second': 4.949, 'train_steps_per_second': 0.619, 'total_flos': 444583053250560.0, 'train_loss': 0.7549880544344584})

## Model Evaluation

In [14]:
# Evaluate on validation set
eval_results = trainer.evaluate()

#  Validation Loss
eval_loss = eval_results["eval_loss"]
print(f"Validation Loss: {eval_loss:.4f}")

#  Validation Perplexity
perplexity = math.exp(eval_loss)
print(f"📉 Validation Perplexity: {perplexity:.2f}")

#  Exact Match Accuracy on Validation Set
def compute_exact_match(model, tokenizer, dataset):
    model.eval()
    correct = 0
    total = 0

    for example in tqdm(dataset, desc="Evaluating EM"):
        input_text = example["text"]
        if "A:" not in input_text:
            continue
        question, true_answer = input_text.split("A:", 1)
        prompt = question.strip() + "A:"

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=50)
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract model's answer
        pred_answer = generated.split("A:")[-1].strip().split("\n")[0]
        true_answer = true_answer.strip().split("\n")[0]

        if pred_answer == true_answer:
            correct += 1
        total += 1

    return correct / total if total > 0 else 0.0

em_score = compute_exact_match(model, tokenizer, val_dataset)
print(f"✅ Exact Match Accuracy on Validation Set: {em_score * 100:.2f}%")

Validation Loss: 0.1686
📉 Validation Perplexity: 1.18


Evaluating EM: 100%|██████████| 100/100 [02:32<00:00,  1.52s/it]

✅ Exact Match Accuracy on Validation Set: 96.00%


In [15]:
print(f"Validation Loss: {eval_loss:.4f}")
print(f"📉 Validation Perplexity: {perplexity:.2f}")
print(f"✅ Exact Match Accuracy on Validation Set: {em_score * 100:.2f}%")

Validation Loss: 0.1686
📉 Validation Perplexity: 1.18
✅ Exact Match Accuracy on Validation Set: 96.00%
